In [1]:
import psycopg2
import pandas as pd
import numpy as np

In [2]:
df =  pd.read_csv(r'C:\Users\DY\Documents\GitHub\UFCproject\CSV Files\df_ufc_masters.csv')

In [3]:
selected_features = [
    'R_fighter',
    'R_avg_SIG_STR_landed',
    'R_avg_SIG_STR_pct',
    'R_avg_SUB_ATT',
    'R_avg_TD_landed',
    'R_avg_TD_pct',
    'R_Height_cms',
    'R_Reach_cms',
    'R_age'
]

In [4]:
df_selected = df[selected_features]

In [5]:
fighter_counts = df_selected['R_fighter'].value_counts().rename('count')

In [6]:
df_selected = df_selected.merge(fighter_counts, left_on='R_fighter', right_index=True)

In [7]:
grouped_features = [
    'R_avg_SIG_STR_landed',
    'R_avg_SIG_STR_pct',
    'R_avg_SUB_ATT',
    'R_avg_TD_landed',
    'R_avg_TD_pct'
]
df_grouped = df_selected.groupby('R_fighter')[grouped_features].mean()

In [8]:
df_final = df_grouped.merge(df_selected[['R_fighter', 'count', 'R_Height_cms', 'R_Reach_cms', 'R_age']].drop_duplicates(), on='R_fighter')

In [9]:
df_final_unique = df_final.drop_duplicates(subset='R_fighter')

In [10]:
# Generating a unique fighter ID
df_final_unique = df_final_unique.reset_index(drop=False)
df_final_unique['fighter_id'] = range(1, len(df_final_unique) + 1)

In [11]:
df_final_unique = df_final_unique.drop(['index'], axis=1)

In [12]:
# Ensure 'fighter_id' is in the DataFrame
if 'fighter_id' in df_final_unique.columns:
    # Reorder columns to make 'fighter_id' the first column
    column_order = ['fighter_id'] + [col for col in df_final_unique.columns if col != 'fighter_id']
    df_final_unique = df_final_unique[column_order]

LOADER INN I SQL

In [13]:
with open('connection_string.txt') as file:
    conn_string = file.readline()

In [14]:
conn = psycopg2.connect(conn_string)
if conn.closed == 0:
    print("Connection established")

Connection established


In [16]:
df_final_unique.columns

Index(['fighter_id', 'R_fighter', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct',
       'R_avg_SUB_ATT', 'R_avg_TD_landed', 'R_avg_TD_pct', 'count',
       'R_Height_cms', 'R_Reach_cms', 'R_age'],
      dtype='object')

In [17]:
cur = conn.cursor()

insert_query = """
    INSERT INTO fighters (fighter_id, R_fighter, R_avg_SIG_STR_landed, R_avg_SIG_STR_pct, R_avg_SUB_ATT, R_avg_TD_landed, R_avg_TD_pct, count, R_Height_cms, R_Reach_cms, R_age)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""
# Prepare a list of tuples from the DataFrame
records_to_insert = [tuple(row) for row in df_final_unique.to_numpy()]

# Execute the query
try:
    cur.executemany(insert_query, records_to_insert)
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print("Error: %s" % error)
    conn.rollback()